# 2) Notebook variants (what to toggle)

Create one notebook per item below. In Cell 4, adjust only the mentioned field:

	1.	Wave speed: A.speed = 1.5, B.speed = 3.2

	2.	Q factor: A.Q = 50, B.Q = 200

	3.	Grid kind: A.grid_kind = “regular”, B.grid_kind = “streams”

(Set CHANNELS_DIR for streams; keep others identical.)

	4.	Wave kind: A.wave_kind = “surface”, B.wave_kind = “body”

	5.	Station corrections switch: A.station_corr = False, B.station_corr = True

(If you have a gains CSV and a hook in asl_sausage, add that there. The template passes None to keep it simple; the goal is just to verify toggling works end-to-end.)

	6.	Distance mode: A.dist_mode = “2d”, B.dist_mode = “3d”

	7.	Misfit engine: A.misfit_engine = “l2”, B.misfit_engine = “huber”

Notes / tips:

	•	These notebooks intentionally avoid shared caches so each run is clean and self-contained for debugging. Once you’re happy, you can wire in your shared cache dirs if you want speed.

	•	If you want the map outputs to always land at the same filename for easy diffing, keep the same PEAKF, Q, etc., and your ASL plotting code will produce deterministically named files (e.g., map_Q50_F8.png).
	
	•	If you want to include the misfit heatmap in these quick tests, make sure your asl_sausage (or the called ASL plotting) writes it (e.g., misfit_heatmap.png). The summary table checks for its existence.

If you want, I can also give you a minimal “compare images side-by-side” cell you can drop in to display map_Q*_F*.png from A and B inline.

In [ ]:
from __future__ import annotations
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import Optional, Dict, Any, Tuple

import os, sys, time, traceback
import numpy as np
import pandas as pd
from obspy import read, read_inventory
from glob import glob

# flovopy bits
from flovopy.core.mvo import dome_location
from flovopy.asl.asl_old import asl_sausage
from flovopy.asl.grid import make_grid, nodegrid_from_channel_csvs, Grid, NodeGrid
from flovopy.asl.distances import compute_or_load_distances, distances_signature
from flovopy.asl.ampcorr import AmpCorr, AmpCorrParams

# plotting helpers
from flovopy.asl.map import topo_map

homedir = os.path.expanduser('~')
projectdir = os.path.join(homedir, 'Dropbox', 'BRIEFCASE', 'SSADenver')
localprojectdir = os.path.join(homedir, 'work', 'PROJECTS', 'SSADenver_local')

originalDEM = os.path.join(homedir, "Dropbox/PROFESSIONAL/DATA/WadgeDEMs/conversions/DEM_1999_WGS84_rotated.tif")
CHANNELSDIR = os.path.join(projectdir, 'channel_finder')
fixedDEM1999 = os.path.join(CHANNELSDIR, '02_dem_flipped_horizontal.tif') # made from tif above, after flipping
GLOBAL_CACHE = os.path.join(localprojectdir, 'asl_global_cache')
invXML = os.path.join(projectdir, 'MV.xml')
MSEED_DIR = os.path.join(projectdir, 'ASL_inputs', 'biggest_pdc_events')
mseedfiles = sorted(glob(os.path.join(MSEED_DIR, '*.cleaned')))

# ==== EDIT THESE ====
MSEED_FILE      = mseedfiles[0]          # one event
INVENTORY_XML   = os.path.join(projectdir, 'MV.xml')  # StationXML
OUTPUT_BASE     = os.path.join(localprojectdir, "asl_notebooks")                # where runs are written
CHANNELS_DIR    = None                                # required if grid_kind="streams"
CHANNELS_STEP_M = 100.0
CHANNELS_DEM    = None                                # optional GeoTIFF for NodeGrid
REGULAR_GRID_DEM = "pygmt:01s"                        # or "geotiff:/abs/path.tif" or None
DEM_TIF_BMAP    = None                                # optional basemap GeoTIFF
NODE_SPACING_M  = 50
MIN_STATIONS    = 5
METRIC          = "VT"                                # or "mean" etc.
WINDOW_SECONDS  = 5
PEAKF           = 8.0
ISLAND_REGION = [-62.255, -62.135, 16.66, 16.84]


In [ ]:
@dataclass(frozen=True)
class MiniConfig:
    grid_kind: str           # 'streams' or 'regular'
    wave_kind: str           # 'surface' or 'body'
    station_corr: bool       # True/False (we won't load a CSV here—just a switch)
    speed: float             # km/s
    Q: int
    dist_mode: str           # '2d' or '3d'
    misfit_engine: str       # 'l2' or 'huber'

    def tag(self) -> str:
        parts = [
            f"G_{self.grid_kind}",
            f"W_{self.wave_kind}",
            f"SC_{'on' if self.station_corr else 'off'}",
            f"V_{self.speed:g}",
            f"Q_{self.Q}",
            f"D_{self.dist_mode}",
            f"M_{self.misfit_engine}",
        ]
        return "__".join(parts)

In [ ]:
# Example baseline that you will tweak per notebook:
A = MiniConfig(grid_kind="regular", wave_kind="surface", station_corr=False,
               speed=1.5, Q=50, dist_mode="2d", misfit_engine="l2")

B = MiniConfig(grid_kind="regular", wave_kind="surface", station_corr=False,
               speed=3.2, Q=50, dist_mode="2d", misfit_engine="l2")  # <-- only one field changes per notebook

In [ ]:
Path(OUTPUT_BASE).mkdir(parents=True, exist_ok=True)

resA = run_single_event(
    A,
    mseed_file=MSEED_FILE,
    inventory_xml=INVENTORY_XML,
    output_base=OUTPUT_BASE,
    node_spacing_m=NODE_SPACING_M,
    metric=METRIC,
    window_seconds=WINDOW_SECONDS,
    peakf=PEAKF,
    channels_dir=CHANNELS_DIR,
    channels_step_m=CHANNELS_STEP_M,
    channels_dem_tif=CHANNELS_DEM,
    regular_grid_dem=REGULAR_GRID_DEM,
    dem_tif_for_bmap=DEM_TIF_BMAP,
)



In [ ]:
resB = run_single_event(
    B,
    mseed_file=MSEED_FILE,
    inventory_xml=INVENTORY_XML,
    output_base=OUTPUT_BASE,
    node_spacing_m=NODE_SPACING_M,
    metric=METRIC,
    window_seconds=WINDOW_SECONDS,
    peakf=PEAKF,
    channels_dir=CHANNELS_DIR,
    channels_step_m=CHANNELS_STEP_M,
    channels_dem_tif=CHANNELS_DEM,
    regular_grid_dem=REGULAR_GRID_DEM,
    dem_tif_for_bmap=DEM_TIF_BMAP,
)

In [ ]:
def flatten(d: Dict[str, Any]) -> Dict[str, Any]:
    return {k: d.get(k) for k in ["tag", "source_csv", "event_qml", "map_png_exists", "misfit_png_exists", "error"]}

df = pd.DataFrame([flatten(resA), flatten(resB)])
df